**Library import**

In [5]:
# Active l'affichage dynamique sur le notebook
import os
import pandas as pd
import matplotlib.pyplot as plt
import math as m
import matplotlib.animation as animation
import numpy as np
%matplotlib qt 
import imageio


In [88]:
import csv
import os

# check all files in the directory
all_files_txt = os.listdir('data/txt/18-05-2024_data_acquisition/')
for filename in all_files_txt:
    if filename.endswith(".txt"):
        with open('data/txt/18-05-2024_data_acquisition/'+filename, 'r') as file:
            lines = file.readlines()

        # Process data
        data = []
        for line in lines:
            if line.strip() and not line.startswith('*'):
                parts = line.strip().split()
                tracker = parts[0].strip(':')
                id = parts[1]
                x = parts[2]
                y = parts[3]
                z = parts[4]
                timestamp = parts[5]
                data.append([tracker, id, x, y, z, timestamp])

        # Write data to a CSV file
        with open('data/csv/18-05-2024/'+filename.replace('.txt', '.csv'), 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['tracker', 'id', 'x', 'y', 'z', 'timestamp'])
            writer.writerows(data)

        print("Data exported in CSV")

Data exported in CSV
Data exported in CSV
Data exported in CSV
Data exported in CSV
Data exported in CSV
Data exported in CSV
Data exported in CSV
Data exported in CSV
Data exported in CSV
Data exported in CSV
Data exported in CSV
Data exported in CSV
Data exported in CSV
Data exported in CSV
Data exported in CSV
Data exported in CSV
Data exported in CSV
Data exported in CSV
Data exported in CSV
Data exported in CSV
Data exported in CSV
Data exported in CSV


Imported data

In [3]:
# Read all csv files in the directory with panda
dataframes = []
for filename in os.listdir('data/csv/18-05-2024/'):
    if filename.endswith(".csv"):
        df = pd.read_csv('data/csv/18-05-2024/'+filename)
        # add a column with the filename
        df['filename'] = filename
        dataframes.append(df)

In [59]:
# Function to calculate time intervals in seconds
def calculate_intervals(timestamps):
    intervals = []
    for i in range(1, len(timestamps)):
        time_diff = (timestamps[i] - timestamps[i-1]) / pd.Timedelta(seconds=1)  # Convert to seconds
        intervals.append(time_diff)
    if len(intervals) > 0:
        intervals.append(intervals.pop(0))  # Move the first interval to the end
    else:
        intervals = [0]  # In case there's only one timestamp
    return intervals

# Ensure output directory exists
output_dir = 'data/gif/18-05-2024/'
os.makedirs(output_dir, exist_ok=True)

# Assume dataframes is a list of your dataframes
for df in dataframes[7:8]:
    data_filtered = df[df['tracker'] == 'Filtered'].reset_index(drop=True)
    data_raweee = df[df['tracker'] == 'Raweee'].reset_index(drop=True)
    
    # Merge data on timestamps to synchronize points
    merged_data = pd.merge_asof(data_raweee.sort_values('timestamp'), data_filtered.sort_values('timestamp'), on='timestamp', suffixes=('_raweee', '_filtered'))

    # Calculate intervals based on the merged timestamps
    combined_timestamps = merged_data['timestamp']
    intervals = calculate_intervals(combined_timestamps)
    print(f"Intervals: {intervals}")

    # Add 0.2 seconds to the last interval
    intervals.append(0.2)

    # Initialize the plot
    fig, ax = plt.subplots()
    ax.set_xlim(-2, 7.5)
    ax.set_ylim(-2, 7.5)
    ax.set_title('Raw vs Filtered Data Plot')
    ax.set_xlabel('X Axis')
    ax.set_ylabel('Y Axis')

    # Create frames
    images = []
    for i in range(len(merged_data)):
        x_raw = merged_data.iloc[i]['x_raweee'] if 'x_raweee' in merged_data.columns else None
        y_raw = merged_data.iloc[i]['y_raweee'] if 'y_raweee' in merged_data.columns else None
        x_fil = merged_data.iloc[i]['x_filtered'] if 'x_filtered' in merged_data.columns else None
        y_fil = merged_data.iloc[i]['y_filtered'] if 'y_filtered' in merged_data.columns else None
        
        if x_raw is not None and y_raw is not None:
            plt.scatter(x_raw, y_raw, color='red', s=30, alpha=0.5, label='Raw' if i == 0 else "")
        if x_fil is not None and y_fil is not None:
            plt.scatter(x_fil, y_fil, color='blue', s=30, alpha=0.5, label='Filtered' if i == 0 else "")

        # Add legend on the first iteration
        if i == 0:
            ax.legend()

        # Pause for the animation based on time difference
        plt.pause(intervals[i])
    
    plt.show()
    plt.close()

Intervals: [0.245722, 0.274146, 0.247152, 0.248661, 0.244353, 0.248323, 0.282786, 0.248471, 0.24333, 0.239789, 0.245779, 0.28004, 3.590951, 0.243512, 0.252216, 0.538674, 0.490278, 0.484745, 4.101546, 0.24792, 0.245857, 0.248596, 0.291172, 0.250139, 0.247247, 7.005764, 0.276776, 0.258805, 0.250796, 0.249188, 0.244501, 0.289589, 0.244435, 0.244242, 0.250385, 6.572048, 0.241601, 0.276778, 0.233562, 0.242777, 0.246109]


In [6]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# Create a figure with two subplots next to each other
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

# Initialize the plots
line_raw, = ax1.plot([], [], 'ro-', label='Raw Data', alpha=0.5)
line_ema, = ax2.plot([], [], 'bo-', label='EMA', alpha=0.5)

# Set plot limits for both subplots
ax1.set_xlim(min(data_n['X']) - 1, max(data_n['X']) + 1)
ax1.set_ylim(min(data_n['Y']) - 1, max(data_n['Y']) + 1)
ax2.set_xlim(min(data_n_filtered['X']) - 1, max(data_n_filtered['X']) + 1)
ax2.set_ylim(min(data_n_filtered['Y']) - 1, max(data_n_filtered['Y']) + 1)

# Set labels and titles for both subplots
ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.set_title('Raw Data Plot')
ax1.legend()

ax2.set_xlabel('X')
ax2.set_ylabel('Y')
ax2.set_title('EMA Plot')
ax2.legend()

# Real points
A1 = [1, 0]
A2 = [1, 3.3]
B1 = [5.5, 0]
B2 = [5.5, 3.3]
O1 = [3.5, 0]
O2 = [3.5, 0.8]
O3 = [3.5, 1.8]
O4 = [3.5, 3.3]

# Plot points on both subplots
for ax in [ax1, ax2]:
    ax.scatter(A1[0], A1[1], c='b', marker='x')
    ax.scatter(A2[0], A2[1], c='b', marker='x')
    ax.scatter(B1[0], B1[1], c='b', marker='x')
    ax.scatter(B2[0], B2[1], c='b', marker='x')
    ax.scatter(O1[0], O1[1], c='b', marker='x')
    ax.scatter(O2[0], O2[1], c='b', marker='x')
    ax.scatter(O3[0], O3[1], c='b', marker='x')
    ax.scatter(O4[0], O4[1], c='b', marker='x')

# Text annotations
texts = [
    ax1.text(A1[0]-0.1, A1[1], 'A1', fontsize=10, ha='right'),
    ax1.text(A2[0]-0.1, A2[1], 'A2', fontsize=10, ha='right'),
    ax1.text(B1[0]-0.1, B1[1], 'B1', fontsize=10, ha='right'),
    ax1.text(B2[0]-0.1, B2[1], 'B2', fontsize=10, ha='right'),
    ax1.text(O1[0]-0.1, O1[1], 'O1', fontsize=10, ha='right'),
    ax1.text(O2[0]-0.1, O2[1], 'O2', fontsize=10, ha='right'),
    ax1.text(O3[0]-0.1, O3[1], 'O3', fontsize=10, ha='right'),
    ax1.text(O4[0]-0.1, O4[1], 'O4', fontsize=10, ha='right')
]

# Initialization function
def init():
    line_raw.set_data([], [])
    line_ema.set_data([], [])
    return line_raw, line_ema

# Animation function
def animate(i):
    x_raw = data_n['X'][:i]
    y_raw = data_n['Y'][:i]
    x_ema = data_n_filtered['X'][:i]
    y_ema = data_n_filtered['Y'][:i]
    line_raw.set_data(x_raw, y_raw)
    line_ema.set_data(x_ema, y_ema)
    for text in texts:
        text.set_zorder(3)  # Ensure text stays on top
    return line_raw, line_ema, *texts

# Create the animation
ani = animation.FuncAnimation(fig, animate, init_func=init, frames=data_n.shape[0], interval=200, blit=True)

# Show the plot with animation
plt.tight_layout()
plt.show()
